In [1]:
import cv2
import numpy as np
import imutils
import glob
import matplotlib.pyplot as plt
import os
import time
from PIL import Image, ImageChops, ImageOps

In [2]:
def Kalman(vars,dim):
    """ Creates a Kalman Filter with given dimensions 
    ...
    Attributes
    ----------
    var : int
    dim : int    
    
    ...
    Returns
    -------
    prediction: np.array(PredVec,1)
    kalman: object of cv2.KalmanFilter(*args)
    """

    PredVec = vars*dim
    # Example: 6,3 Which states for position and velocity in 3D (x,y,z,vx,vy,vz)
    
    kalman = cv2.KalmanFilter(PredVec, dim) 

    # Define Measurement Matrix: Z
    kalman.measurementMatrix = np.array([[1,0,0,0,0,0],             
                                         [0,0,1,0,0,0],
                                         [0,0,0,0,1,0]], np.float32)

    # Define Transiiton Matrix: F
    kalman.transitionMatrix = np.array([[1,1,0,0,0,0],              
                                        [0,1,0,0,0,0],
                                        [0,0,1,1,0,0],
                                        [0,0,0,1,0,0],
                                        [0,0,0,0,1,1],
                                        [0,0,0,0,0,1]], np.float32)
                                        
    # Define Statistical Noises
    kalman.processNoiseCov = np.eye(6, dtype=np.float32) * 0.05
    kalman.measurementNoiseCov = np.eye(3, dtype=np.float32) * 0.12

    prediction = np.zeros((PredVec,1), np.float32)

    return prediction, kalman

In [3]:
def BkgRmv(sel=1):
    """ This function selects the background subtraction algorithm on the image. 
    ...
    Attributes
    ----------
    sel: int
        select the algorthm
        0:  KNN
        1:  GMM 
        2:  Gemoetric Multigrid
        3:  Frame Difference

    ...
    Returns
    -------
    fgbg : object
    """

    if sel == 0:
        print("BackGroundSubstractor KNN")
        fgbg = cv2.createBackgroundSubtractorKNN()#history=1000, dist2Threshold=1000, detectShadows=True)
    elif sel== 1:
        print("BackGroundSubstractor Gaussian mixture model")
        fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
    elif sel == 2:
        print(" BackGroundSubstractor Geometric Multigrid")
        print(" NEEDS TO BEINSTALLED SEPARATEDLY")
        fgbg = cv2.bgsegm.createBackgroundSubtractorGMG()
    else: 
        print("Frame Difference method")
        """"
        if frameNum == 1:
    	bgFrame = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
        elif frameNum > 1:
            foreFrame = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
            foreFrame = cv2.absdiff(foreFrame, bgFrame)
            _, thresh = cv2.threshold(foreFrame, 30, 255, cv2.THRESH_BINARY)
            gaussian = cv2.GaussianBlur(thresh, (3, 3), 0)
            cv2.imshow('gaussian', foreFrame)
        """""
    return fgbg

In [4]:
def sum_abs_diff(image1, image2):
    """ This function calculates the sum of absolute difference between an image pair. 
    ...
    Attributes
    ----------
    image1: np.array()
    image2: np.array()

    ...
    Returns
    -------
    sad : float
    """

    image1 = image1.astype('int32')
    image2 = image2.astype('int32')
    
    sad = 0 #Sum of Absolute Differences
    
    if image1.shape == image2.shape:
        diff = image1 - image2
        sad = np.sum(np.absolute(diff))
    else:
        sad = -1
    
    return sad

def scan_line(span, template, search_col_min, search_col_max):
    """ This function scans along the epipolar line to find the 
        mathcing points based on sum of absolute differenes for 
        the stereo image pair.
    ...
    Attributes
    ----------
    span: int
    template: np.array()
    search_col_min: int
    search_col_max: int

    ...
    Returns
    -------
    (min_place, min_value): tuple(int)
    """
    min_place = -1
    min_value = float('inf')
    for i in range(search_col_min, search_col_max):
        diff = sum_abs_diff(span[:, i:i + span.shape[0]], template)
        if diff < min_value:
            min_value = diff
            min_place = i
    return (min_place, min_value)

def Depth(leftGrayImg, rightGrayImg, positionOnLeft, halfWindow):
    """ This function calculates the deisparity and depth for a stereo 
        image pair based on epipolar geometry and sum of abosulute 
        differences. 
    ...
    Attributes
    ----------
    leftGrayImg: np.array()
    rightGrayImg: np.array()
    postionOnLeft: np.array()
    halfWindow: int

    ...
    Returns
    -------
    depth : float
    """
    baseline = 120
    focalLength = 700
    
    template = leftGrayImg[positionOnLeft[1]-halfWindow:positionOnLeft[1]+halfWindow,\
                           positionOnLeft[0]-halfWindow:positionOnLeft[0]+halfWindow]    
    span = rightGrayImg[positionOnLeft[1]-halfWindow:positionOnLeft[1]+halfWindow, :]
    min_place, min_value = scan_line(span, template, positionOnLeft[0]-250, positionOnLeft[0]-50-halfWindow)
    
    if min_place < 0:
        print('fail')
        
    disparity = positionOnLeft[0]-min_place
    #print(f"Disparity: {disparity} pixels")
    
    depth = focalLength*baseline/disparity
    #print(f"Depth of pixel [{positionOnLeft[0]},{positionOnLeft[1]}] in mm: {depth}")
    return depth

In [10]:
def load_calib_params(path_to_calib_params = "./../Data/calibration_params/"):
    """ This function loads the saved camera calibration paramters."""
    
    mtx_l = np.loadtxt(os.path.join(path_to_calib_params, 'mtx_l.csv'), delimiter=',')
    dist_l = np.loadtxt(os.path.join(path_to_calib_params, 'dist_l.csv'), delimiter=',')
    mtx_r = np.loadtxt(os.path.join(path_to_calib_params, 'mtx_r.csv'), delimiter=',')
    dist_r = np.loadtxt(os.path.join(path_to_calib_params, 'dist_r.csv'), delimiter=',')
    R = np.loadtxt(os.path.join(path_to_calib_params, 'R.csv'), delimiter=',')
    T = np.loadtxt(os.path.join(path_to_calib_params, 'T.csv'), delimiter=',')

    return mtx_l, dist_l, mtx_r, dist_r, R, T

mtx_l, dist_l, mtx_r, dist_r, R, T = load_calib_params(path_to_calib_params = "./../Data/calibration_params/")

In [11]:
def rectify_image_pair(img_l, img_r, mtx_l, dist_l, mtx_r, dist_r, R, T):
    """ This function employs rectification and undistortion of the stereo
        image pair based on given camera calibration parameters. 
    ...
    Attributes
    ----------
    img_l: np.array()
    img_r: np.array()
    mtx_l: np.array()
    dist_l: int
    mtx_r: np.array()
    dist_r: int    
    R: np.array()
    T: np.array()
    ...
    Returns
    -------
    Rectified and Undistorted Stereo Image Pair,
    img_l: np.array()
    img_r: np.array()
    """
    size = np.shape(img_l)[:2]
    size = size[::-1]
    
    R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(mtx_l, dist_l, mtx_r, dist_r, size, R, T, 
                                                        flags = cv2.CALIB_ZERO_DISPARITY, alpha=0.55)
    
    mapl1, mapl2 = cv2.initUndistortRectifyMap(mtx_l, dist_l, R1, P1, (np.shape(img_l)[:2])[::-1], cv2.CV_32FC1)
    mapr1, mapr2 = cv2.initUndistortRectifyMap(mtx_r, dist_r, R2, P2,(np.shape(img_r)[:2])[::-1], cv2.CV_32FC1)
    
    iml = cv2.remap(img_l, mapl1, mapl2, cv2.INTER_LINEAR)
    imr = cv2.remap(img_r, mapr1, mapr2, cv2.INTER_LINEAR)
    
    iml = iml[60:512, 300:]
    imr = imr[60:512, 300:]
    iml = cv2.resize(iml, [1280, 720], interpolation = cv2.INTER_AREA)
    imr = cv2.resize(imr, [1280, 720], interpolation = cv2.INTER_AREA)

    return iml, imr

In [15]:
last_x =0
last_y =0
under_occlusion = False

In [43]:
# Video Buffer
video_buffer = []
bg_buffer = []
font = cv2.FONT_HERSHEY_COMPLEX

#SELECTORS. Preprocess image?
morph = True
crop = True
distortion = 1 # 0: Standard,  1: Rectified, 2: Undistorted
sel = 1 # 0 is KNN, 1 is MOG2 and 2 is MANUAL

#Load Video Frames
rightSet = sorted(glob.glob('./../Data/Stereo_conveyor_with_occlusions_undistorted/right/*.png')) #= cv2.VideoCapture(CamL_id)

#Create Kalman Filter for state prediction
prediction, kalman = Kalman(2,3)

#Create Background Removal
fgbg = BkgRmv(sel)

# Criteria for New Object?
thresholdCenter = 10
prevCenter = (0, 0)
prevRadius = 0
thresholdRadius = 5
tresholdArea = 5
prev_area = 0
rep = 0
kernel = np.ones((5,5), np.uint8)

# Crop the image?
cropTop = 280
cropBottom = 600
cropLeft = 240
cropRight = 1070
frame = 0 

vX=[]
vY=[]

# Loop throiugh the frames
for rightName in rightSet:
    frame += 1
    printed = False
    print("Next Frame")
    leftName = rightName.replace('right','left').replace('Right','Left')
    
    ######################################################################
    #                         IMAGE SELECTION
    ######################################################################
    if sel ==2:
        imgright = Image.open(rightName)
        imgleft = Image.open(leftName)
    else:
        imgright = cv2.imread(rightName)
        imgleft = cv2.imread(leftName)
    if distortion == 0 :
        imrght = cv2.cvtColor(imgright,cv2.COLOR_BGR2GRAY)
        imlft = cv2.cvtColor(imgleft, cv2.COLOR_BGR2GRAY)
    elif distortion == 1 : #Attempt with rectified. TOO MUCH FOV LOSS
        imgleft, imgright = rectify_image_pair(imgleft, imgright, mtx_l, dist_l, mtx_r, dist_r, R, T)
        imrght = cv2.cvtColor(imgright,cv2.COLOR_BGR2GRAY)
        imlft = cv2.cvtColor(imgleft, cv2.COLOR_BGR2GRAY)
    elif distortion == 2 : #Attempt with undistortion. A BIT BETTER, NOT ENOUGH
        imgleft = cv2.undistort(imgleft, mtx_l, dist_l, None, None)
        imgright = cv2.undistort(imgright, mtx_r, dist_r, None, None)
        imrght = cv2.cvtColor(imgright,cv2.COLOR_BGR2GRAY)
        imlft = cv2.cvtColor(imgleft, cv2.COLOR_BGR2GRAY)
    else:
        print("Distorion selector issue")
    originalFrame = imgright
    
    if morph == True and sel != 2:
        ######################################################################
        #                   MORPHOLOGICAL OPS. REDUCE NOISE 
        ######################################################################
        # Smoothening of the image 
        # Morphological Operations: Erosion +  Dilation
        kernel = np.ones((5,5), np.uint8) 
        imrght = cv2.morphologyEx(imrght,cv2.MORPH_OPEN,kernel)
        imrght = cv2.morphologyEx(imrght,cv2.MORPH_OPEN,kernel)  #Open = Erosion + Dilation
    
    if sel == 2:
        ######################################################################
        #                         MANUAL FILTER 
        ######################################################################
        if frame == 1:
            imgright = Image.open(rightName)
            img_empty = imrght
            continue
        else:
            diff = ImageChops.difference(img_empty, imrght)
            diff_np = np.array(diff)
            ret,thresh1 = cv2.threshold(diff_np,40,255,cv2.THRESH_BINARY) #Changed threshold
            kernel = np.ones((10,10), np.uint8)
            img_erosion = cv2.erode(np.asarray(thresh1), kernel, iterations = 1)
            img_dilation = cv2.dilate(img_erosion, kernel, iterations = 3)
            filtered = cv2.morphologyEx(img_dilation,cv2.MORPH_OPEN,kernel)
    else:
        ######################################################################
        #                   BACKGROUND SUBSTRACTION & MORPH 
        ######################################################################
        kernel = np.ones((5,5), np.uint8)
        filtered = fgbg.apply(imrght)
        #imrght = cv2.morphologyEx(imrght,cv2.MORPH_OPEN,kernel)
        filtered = cv2.erode(filtered, kernel, iterations = 3)
        filtered = cv2.dilate(filtered, kernel, iterations = 4)
        filtered[380:720, 560:1280] = 0
        filtered[0:400, 560:880] = 0
            
        
    #####################################################################
    #                   CONTOURS AFTER FILTERING
    ######################################################################
    #Focus the analysis in the area of interest
    cropped = filtered[cropTop:cropBottom, cropLeft:cropRight]
    cnts, _ = cv2.findContours(cropped.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if len(cnts) > 0:
        under_occlusion = False
        c = max(cnts, key=cv2.contourArea)   #Select biggest contour based on area criteria
        area = cv2.contourArea(c)            #Obtain its area
        M = cv2.moments(c)                   #Find the center of the shape
        if  not M["m00"] == 0.0 and (area<25000) and (area>500) :
            
            cX = int(M["m10"] / M["m00"]) + cropLeft 
            cY = int(M["m01"] / M["m00"]) + cropTop
            center = (cX,cY)
            depth = Depth(imlft, imrght, center, 50)
            cv2.circle(originalFrame, center, 3, (0, 0, 255), -1)
            cv2.circle(originalFrame, center, 7, (0, 0, 255), 2)
            x,y,w,h = cv2.boundingRect(c)        #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.rectangle(originalFrame,(x+cropLeft,y+cropTop),(x+cropLeft+w,y+h+cropTop),(0,255,0),2)
            # cv2.putText(originalFrame, f"Depth: {depth:.2f} mm", (0,200), cv2.FONT_ITALIC, 1, (0,0,0))
                     
            ###########################################
            #         Condition Regarding Area
            ###########################################
            if cX > 1047:
                prediction, kalman = Kalman(2,3)
                vX.clear()
                vY.clear()
                last_x = cX
                last_y = cY

            if (prev_area*0.8) >= area and cX > 860:
                rep+=1
                if rep >= 4:
                    cv2.putText(originalFrame,"Losing object", (0, 150), font, 1, (0, 0, 255))
            else:
                # Correct kalman 
                rep = 0
                if (cX<1040 and cX>955):
                    if cX*0.97<=last_x and last_y<=cY*1.03: #and cX >= last_x-15 and cY <= last_y+15:
                        kalman.correct(np.array([np.float32(cX), np.float32(cY), np.float32(depth)], np.float32))
                        last_x = cX
                        last_y = cY
                        prediction = kalman.predict()
                        kalman.correct(np.array([np.float32(cX-2), np.float32(cY+4), np.float32(depth)], np.float32))
                        cv2.putText(originalFrame,"TRAINING KALMAN", (0, 150), font, 1, (0, 255, 0))
                        if cX<1040 and cX>955:
                            vX.append(int(prediction[1]))
                            vY.append(int(prediction[3]))
                            cv2.putText(originalFrame,"STORING SPEED", (0, 100), font, 1, (0, 255, 255))
                elif cX < 955 and cX > 880: 
                    estimatedX = last_x - np.mean(vX)*0.15
                    estimatedY = last_y + np.mean(vY)*0.15
                    kalman.correct(np.array([np.float32(estimatedX), np.float32(estimatedY), np.float32(depth)], np.float32))
                    last_x = estimatedX
                    last_y = estimatedY
                    cv2.putText(originalFrame,"INTERPOLATING KALMAN", (0, 175), font, 1, (0, 255, 255))
            
            if cX < 520:
                kalman.correct(np.array([np.float32(cX), np.float32(cY), np.float32(depth)], np.float32))
                prev_area = area   

    else:
        # Object not found
        text = "Object not found"
        objectFound = False
        colour = (0, 0, 255)
        cv2.putText(originalFrame, text, (0, 50), font, 1, colour) 
        under_occlusion = True           

    prediction = kalman.predict()
    if prediction[0]<320 or prediction[2]>640:
        prediction[0]=300
        prediction[2]=560
    if prediction[0] >= 300:
        cv2.circle(originalFrame, (int(prediction[0]), int(prediction[2])), 5, (255, 0, 0), -1)

    if under_occlusion:
        depth = 0                            

    cv2.putText(originalFrame, f"Estimates: {prediction[0]} mm, {prediction[2]} mm, {prediction[4]} mm", \
                                (0,300), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0))
    cv2.putText(originalFrame, f"Depth: {depth:.2f} mm", (0,220), font, 1, (42,252,217))

    resized1 = cv2.resize(originalFrame, (640,360), interpolation = cv2.INTER_AREA)
    resized2 = cv2.resize(filtered, (640,360), interpolation = cv2.INTER_AREA)
    cv2.imshow('Tracking', resized1)
    cv2.imshow('Background Subtraction', resized2)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break

    video_buffer.append(resized1)
    bg_buffer.append(resized2)

   

BackGroundSubstractor Gaussian mixture model
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame


/Users/invisible_man/opt/anaconda3/envs/Perception/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/invisible_man/opt/anaconda3/envs/Perception/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame

In [39]:
out = cv2.VideoWriter('./../Output/tracking.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, (video_buffer[1].shape[1], video_buffer[1].shape[0]))
bg_out = cv2.VideoWriter('./../Output/bg_sub.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, (bg_buffer[1].shape[1], bg_buffer[1].shape[0]))
 
for i in range(len(video_buffer)):
    out.write(video_buffer[i])
out.release()

for i in range(len(bg_buffer)):
    bg_out.write(bg_buffer[i])
bg_out.release()


In [42]:
for i in range(len(bg_buffer)):
    cv2.imshow(bg_buffer[i])
    if cv2.waitKey(30) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break


error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)
